# 1. 데이터 로드

In [1]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq
import models.google.google_trend as google_trend
import nest_asyncio
import asyncio

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:
srch_keyword

['keyword_final']

In [6]:
import os
import csv
import datetime
import asyncio
import pandas as pd

# 필요한 경우 비동기를 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

from collect_keywords import collect_keywords

async def main(srch_keyword, day):
    # 오늘 날짜로 폴더 경로 생성
    folder_path = './data/rl_srch/' + datetime.datetime.now().strftime('%y%m%d')
    file_path = f"{folder_path}/collected_keywords.csv"
    
    # 폴더가 존재하는지 확인
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 존재하는지 확인
    if os.path.isfile(file_path):
        # 파일이 존재하면, 데이터를 읽어옵니다.
        collected_keywords_data = pd.read_csv(file_path)
    else:
        # 파일이 없으면, collect_keywords 함수를 호출해서 데이터를 수집합니다.
        collected_keywords_data = await collect_keywords(srch_keyword, day)
        # 결과를 CSV로 저장
        collected_keywords_data.to_csv(file_path, index=False)
    
    return collected_keywords_data
collected_keywords_data=asyncio.run(main(srch_keyword, day))


In [7]:

collected_keywords_dat_copy=asyncio.run(main(srch_keyword, day))

In [8]:
sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)

# 각 '검색어'별로 분리된 DataFrame을 리스트에 저장
grouped = sorted_df.groupby('검색어')
df_list = [group for _, group in grouped]
print(len(df_list))

45


C:\Users\chohy\AppData\Local\Temp\ipykernel_12148\3376418344.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)


In [9]:
from utils import merge_and_mark_duplicates_limited
collected_keywords_data = merge_and_mark_duplicates_limited(df_list)


collected_keywords_data.head(5)



,연관키워드,월간검색수_합계,검색어,중복검색어
0,달러환율,2026500.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식"
1,적금계산기,636400.0,CMA금리비교,"CMA금리비교,CMA통장"
2,다우지수,538800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주"
3,적금,464000.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식"
4,코스닥,421000.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매..."


In [10]:
import utils

def add_client_info(collected_keywords_data, start_id_index=1):
    clients = utils.get_secret("clients")
    start_id_index = 1
    clients = utils.get_secret("clients")
    # ID와 PW 컬럼을 데이터프레임에 추가하는 로직
    total_rows = len(collected_keywords_data)
    ids = []
    pws = []

    for i in range(total_rows):
        # 현재 id 인덱스 계산 (start_id_index를 기준으로)
        current_id_index = ((i // 500) + start_id_index) % len(clients)
        current_id_key = f"id_{current_id_index}"
        
        # 현재 id와 pw 할당
        current_id = clients[current_id_key]['client_id']
        current_pw = clients[current_id_key]['client_secret']
        
        ids.append(current_id)
        pws.append(current_pw)

    # ID와 PW 컬럼 추가
    collected_keywords_data['id'] = ids
    collected_keywords_data['pw'] = pws

    return collected_keywords_data
collected_keywords_data= add_client_info(collected_keywords_data)
new_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성']

for column in new_columns:
    collected_keywords_data[column] = 0
collected_keywords_data.head(5)

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw,일별급상승,주별급상승,월별급상승,주별지속상승,월별지속상승,월별규칙성
0,달러환율,2026500.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
1,적금계산기,636400.0,CMA금리비교,"CMA금리비교,CMA통장",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
2,다우지수,538800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
3,적금,464000.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
4,코스닥,421000.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0


In [11]:
def groupped_df(name,collected_keywords_data):
    grouped = collected_keywords_data.groupby(name)
    df_list = [group for _, group in grouped]
    return df_list
df_list=groupped_df('id',collected_keywords_data)
print(len(df_list))

3


In [12]:
import asyncio
import trend 
# 비동기 메인 함수 수정
async def trend_main(df, clients):
    # 파라미터 설정
    params = {
        "search_keywords": list(df['연관키워드']),
        "id": df['id'].iloc[0],
        "pw": df['pw'].iloc[0],
        "api_url": "https://openapi.naver.com/v1/datalab/search",
        "name": '연관검색어'
    }
    api_url = "https://openapi.naver.com/v1/datalab/search"
    
    # trend_maincode 함수 실행
    results = await trend.trend_maincode(params, clients, api_url)
    return results

async def run_all(df_list, clients):
    tasks = [trend_main(df, clients) for df in df_list]
    results = await asyncio.gather(*tasks)
    return results

clients = get_secret("clients")  # clients 정보를 로드
# 이벤트 루프 실행

trend_main_data = asyncio.run(run_all(df_list, clients))
results =trend_main_data.copy()

In [36]:
from select_keyword import select_keyword, rising_keyword_analysis, monthly_rule

start_time = time.time()
select_periods = ['daily', 'weekly', 'month']
rising_periods=['weekly', 'month']

formatted_today, today_date = utils.get_today_date()
month_rule_list=[]
select_list=[[],[],[]]

rising_list=[[],[]]
rising_month_list=[]
# 월별, 주별, 일별 키워드 분석 실행

    # 각 분석 기간에 대해 결과 집합을 순회합니다.
for keyword_group in results:
    # 키워드 그룹의 각 키워드 데이터프레임에 대해 순회합니다.
    for keyword_data in keyword_group:
        # 월별 규칙을 적용하여 결과를 가져옵니다.
        monthly_data, monthly_chart, similarity_rate, rising_months = monthly_rule(keyword_data, today_date, 'month')
        
        if monthly_data is not None:
            # 결과 데이터프레임의 열 이름을 가져옵니다.
            column_names = monthly_data.columns
            rising_month_list.append([rising_months,column_names[0]])
            # 결과 데이터프레임에서 값 리스트를 추출합니다.
            data_values_list = monthly_data[column_names].values
            # 월별 차트에 데이터 값을 추가합니다.
            monthly_chart['Indicator'] = data_values_list
            monthly_chart['InfoData'] = similarity_rate
            # 상승 월 정보를 추가합니다. 상승 월이 없는 경우 0으로 설정합니다.
            monthly_chart['RisingMonth'] = 0
            
            # 최종 결과 리스트에 수정된 월별 차트를 추가합니다.
            month_rule_list.append(monthly_chart)
                
# 주별, 월별 상승 키워드 분석 실행
rising_analysis_periods = ['weekly', 'month']
i=0
for period in rising_analysis_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            rising_tmp, rising_graph, rising_info = rising_keyword_analysis(keyword_df, today_date, period)
            if rising_tmp is not None:
                column_names=rising_tmp.columns
                data_values_list = rising_tmp[column_names].values
                rising_graph['Indicator'] = data_values_list
                rising_graph['InfoData'] = rising_info

                rising_list[i].append(rising_graph)
    i=i+1

    


i = 0
# 일별, 주별, 월별 키워드 선택 실행
for period in select_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            selected_tmp, selected_graph, selected_info = select_keyword(keyword_df, today_date, period)
            if selected_graph is not None:
                # 데이터프레임의 열 이름을 출력합니다.
                selected_graph['InfoData'] = selected_info
                select_list[i].append(selected_graph)
            else:
                pass
    i += 1

end_time = time.time()
print(f"Analysis completed in {end_time - start_time} seconds.")

월별 규칙성 키워드 발견 : 데이트앱
[2, 3, 5, 6, 8, 10, 12]
월별 규칙성 키워드 발견 : 거실벽인테리어
[1, 2, 3, 6, 8, 10, 11, 12]
월별 규칙성 키워드 발견 : 동물그림
[1, 2, 3, 5, 6, 8, 10]
월별 규칙성 키워드 발견 : 석상
[1, 2, 3, 4, 6, 8, 11]
월별 규칙성 키워드 발견 : 명함
[1, 2, 3, 6, 8, 10]
월별 규칙성 키워드 발견 : 명함제작
[1, 2, 3, 6, 8, 10]
월별 규칙성 키워드 발견 : 주택관리사
[2, 3, 5, 6, 8, 10, 11]
월별 규칙성 키워드 발견 : 월세
[1, 2, 3, 4, 6, 10, 12]
월별 규칙성 키워드 발견 : 상가임대
[1, 2, 3, 5, 6, 10]
월별 규칙성 키워드 발견 : 부동산매매
[1, 2, 3, 4, 6, 10]
월별 규칙성 키워드 발견 : 상속세세율
[1, 2, 3, 4, 5, 6, 8, 9]
월별 규칙성 키워드 발견 : 상속재산분할협의서
[1, 2, 3, 5, 6, 9, 10, 11]
월별 규칙성 키워드 발견 : 상속포기신고서
[1, 2, 3, 5, 6, 10]
월별 규칙성 키워드 발견 : 상속포기신청
[2, 3, 5, 6, 8, 9]
월별 규칙성 키워드 발견 : 연말정산간소화
[1, 2, 3, 4, 5, 8, 9, 10, 11, 12]
월별 규칙성 키워드 발견 : 부가세
[1, 2, 3, 4, 6, 8, 9, 10, 12]
월별 규칙성 키워드 발견 : 회계
[2, 3, 6, 8, 10, 11]
월별 규칙성 키워드 발견 : 법무사
[1, 2, 3, 5, 6, 8, 10]
월별 규칙성 키워드 발견 : 차용증양식
[1, 2, 3, 6, 8, 10, 11]
월별 규칙성 키워드 발견 : 전세권설정
[1, 2, 3, 4, 6, 8, 10]
월별 규칙성 키워드 발견 : 등기
[1, 2, 3, 6, 8, 10, 11]
월별 규칙성 키워드 발견 : 아파트월세
[1, 2, 3, 6, 8, 10, 12]
월별 규칙성 키

In [38]:
def update_keywords_flag(dataframe, data_list, flag_name):
    unique_associated_keywords = set()
    for df in data_list:
        unique_associated_keywords.update(df['연관검색어'].unique())
    for index, row in dataframe.iterrows():
        if row['연관키워드'] in unique_associated_keywords:
            dataframe.at[index, flag_name] = 1

flags_and_lists = [
    ("일별급상승", select_list[0]),
    ("주별급상승", select_list[1]),
    ("월별급상승", select_list[2]),
    ("주별지속상승", rising_list[0]),
    ("월별지속상승", rising_list[1]),
    ("월별규칙성", month_rule_list),
]

for flag_name, data_list in flags_and_lists:
    update_keywords_flag(collected_keywords_data, data_list, flag_name)


In [39]:
collected_keywords_data

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw,일별급상승,주별급상승,월별급상승,주별지속상승,월별지속상승,월별규칙성
0,달러환율,2026500.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
1,적금계산기,636400.0,CMA금리비교,"CMA금리비교,CMA통장",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
2,다우지수,538800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
3,적금,464000.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
4,코스닥,421000.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",y3ft_k6LA9fg4OYuuqKz,pHm0b5broI,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1138,S&P500지수,32150.0,테마주,"테마주,퇴직연금",PpaBjazJm1ErL0ecFR1i,OZK7n9HJN3,0,0,0,0,0,0
1139,항셍지수,32070.0,테마주,테마주,PpaBjazJm1ErL0ecFR1i,OZK7n9HJN3,0,0,0,0,0,0
1140,코스피야간선물,30190.0,테마주,테마주,PpaBjazJm1ErL0ecFR1i,OZK7n9HJN3,0,0,0,0,0,0
1141,양자컴퓨터관련주,30170.0,테마주,테마주,PpaBjazJm1ErL0ecFR1i,OZK7n9HJN3,0,0,0,0,0,0


구글이나 네이버 merge하면 끝 근데 상승월이랑 활동성 컬럼 추가해야댐

In [40]:
month_rule_list

[          기준일자     유형 연관검색어       검색일자         검색량   Indicator  InfoData  \
 0   2024-03-11  월별급상승  데이트앱 2021-07-04    0.000000   59.348361     90.24   
 1   2024-03-11  월별급상승  데이트앱 2021-08-01    0.000000   61.904752     90.24   
 2   2024-03-11  월별급상승  데이트앱 2021-08-29    0.000000   66.015025     90.24   
 3   2024-03-11  월별급상승  데이트앱 2021-09-26    0.000000   74.536331     90.24   
 4   2024-03-11  월별급상승  데이트앱 2021-10-24    0.000000   78.045106     90.24   
 5   2024-03-11  월별급상승  데이트앱 2021-11-21    0.000000   67.669170     90.24   
 6   2024-03-11  월별급상승  데이트앱 2021-12-19    0.000000   74.586456     90.24   
 7   2024-03-11  월별급상승  데이트앱 2022-01-16    0.000000   71.829563     90.24   
 8   2024-03-11  월별급상승  데이트앱 2022-02-13    0.000000  100.000000     90.24   
 9   2024-03-11  월별급상승  데이트앱 2022-03-13    0.000000   89.674188     90.24   
 10  2024-03-11  월별급상승  데이트앱 2022-04-10    0.000000   82.255636     90.24   
 11  2024-03-11  월별급상승  데이트앱 2022-05-08    0.000000   97.844612     90.24   

In [41]:
import pandas as pd
def process_data(data, condition, type_label, data_lists):
    # 조건에 맞는 데이터 필터링
    filtered_data = data[data[condition] == 1].copy()
    filtered_data['유형'] = type_label
    
    # 불필요한 열 삭제
    columns_to_drop = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성', 'id', 'pw', '검색어']
    filtered_data.drop(columns_to_drop, axis=1, inplace=True)
    
    # 인덱스 재설정
    filtered_data.reset_index(drop=True, inplace=True)
    
    # '지표' 열 초기화
    filtered_data['지표'] = None  
    
    # '지표' 열에 데이터 채우기
    for i, df in enumerate(data_lists):
        if i < len(filtered_data):  
            filtered_data.at[i, '지표'] = str(df['InfoData'].iloc[0]) + '%' 
    # '상승월' 열 추가
    filtered_data['상승월'] = None
    
    return filtered_data

info_result_daily_select = process_data(collected_keywords_data, '일별급상승', '일별급상승', select_list[0])
info_result_weekly_select = process_data(collected_keywords_data, '주별급상승', '주별급상승', select_list[1])
info_result_monthly_select = process_data(collected_keywords_data, '월별급상승', '월별급상승', select_list[2]) 

info_result_weekly_continuous = process_data(collected_keywords_data, '주별지속상승', '주별지속상승', rising_list[0])

info_result_monthly_continuous = process_data(collected_keywords_data, '월별지속상승', '월별지속상승', rising_list[1])

info_result_monthly_pattern = process_data(collected_keywords_data, '월별규칙성', '월별규칙성', month_rule_list)

info_result_final = pd.concat([info_result_daily_select,info_result_weekly_select, info_result_monthly_select,\
                               info_result_weekly_continuous, info_result_monthly_continuous,\
                                  info_result_monthly_pattern]).reset_index(drop=True)
info_result_final

,연관키워드,월간검색수_합계,중복검색어,유형,지표,상승월
0,중고차어플,3470.0,"돈버는앱,리워드",일별급상승,280.42%,None
1,아이큐코인,34060.0,암호화폐,일별급상승,15.25%,None
2,도지코인시세,137240.0,CMA금리비교,주별급상승,35.4%,None
3,레고켐바이오주가,80820.0,"CMA금리비교,채권금리",주별급상승,12.8%,None
4,트레이딩뷰,78900.0,"CMA금리비교,WTI,급등주,미국주식,암호화폐,채권,채권금리,테마주",주별급상승,22.77%,None
...,...,...,...,...,...,...
617,공모전,38300.0,조각투자,월별규칙성,93.6%,None
618,솔루션,21420.0,조각투자,월별규칙성,90.11%,None
619,소득공제용연금저축,15130.0,중개형ISA,월별규칙성,90.76%,None
620,직계존속,32300.0,증여,월별규칙성,93.11%,None


## Graph_result_df 

In [42]:
def select_and_process(df, condition, type_label):
    # 조건에 맞는 데이터 선택
    selected_df = df[df[condition] == 1].copy()
    selected_df['유형'] = type_label
    
    # 불필요한 열 삭제
    drop_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성', 'id', 'pw', '검색어', '월간검색수_합계', '중복검색어']
    selected_df.drop(drop_columns, axis=1, inplace=True)
    
    # 인덱스 초기화
    selected_df.reset_index(drop=True, inplace=True)
    
    return selected_df

# 각 조건에 대해 함수 호출
conditions_and_labels = [
    ('일별급상승', '일별급상승'),
    ('주별급상승', '주별급상승'),
    ('월별급상승', '월별급상승'),
    ('주별지속상승', '주별지속상승'),
    ('월별지속상승', '월별지속상승'),
    ('월별규칙성', '월별규칙성'),
]

# 빈 리스트를 생성하여 각 조건에 대한 결과를 저장
selected_dfs = []

for condition, label in conditions_and_labels:
    selected_df = select_and_process(collected_keywords_data, condition, label)
    selected_dfs.append(selected_df)

# 필요에 따라 selected_dfs 리스트에서 각 데이터프레임에 접근할 수 있습니다.
# 예: daily_select_df = selected_dfs[0]

concatenated_df = pd.concat(selected_dfs, ignore_index=True)


In [43]:
# 결과를 저장할 빈 리스트 초기화
matching_indices = []

# results 리스트를 반복하며 각 데이터프레임 탐색
for i, inner_list in enumerate(results):
    for j, df in enumerate(inner_list):
        asset_name = df.columns[0]  # 각 데이터프레임의 열 이름 (가상자산 이름)
        
        # concatenated_df에서 asset_name과 일치하는 연관키워드를 찾고, 해당 유형을 가져옴
        matching_rows = concatenated_df[concatenated_df['연관키워드'] == asset_name]
        
        # 일치하는 연관키워드가 있다면
        if not matching_rows.empty:
            for _, row in matching_rows.iterrows():
                type_name = row['유형']  # 일치하는 유형 이름
                # 결과 리스트에 인덱스와 유형을 추가
                matching_indices.append([i, j, type_name])


# 초기 df_result DataFrame을 생성합니다. 이는 나중에 각 결과를 쌓기 위한 빈 DataFrame입니다.
import pandas as pd
from datetime import datetime, timedelta

# matching_indices와 results가 정의되어 있다고 가정
graph_result = pd.DataFrame()

for idx in matching_indices:
    temp_df = results[idx[0]][idx[1]].reset_index()
    name = temp_df.columns[1]
    temp_df['연관검색어'] = name
    temp_df.rename(columns={'date': '검색일자', name: '검색량'}, inplace=True)
    temp_df['유형'] = idx[2]
    temp_df['오늘일자'] = datetime.today().strftime('%Y-%m-%d')
    
    # '검색일자'를 datetime 객체로 변환
    temp_df['검색일자'] = pd.to_datetime(temp_df['검색일자'])
    
    # 유형에 따라 데이터 필터링
    if '주별' in temp_df['유형'].iloc[0]:
        # 주별 데이터: 7일 간격으로 필터링
        temp_df = temp_df[temp_df.index % 7 == 0]
    elif '월별' in temp_df['유형'].iloc[0]:
        # 월별 데이터: 28일 간격으로 필터링
        temp_df = temp_df[temp_df.index % 28 == 0]
    
    graph_result = pd.concat([graph_result, temp_df], ignore_index=True)

# graph_result 데이터프레임의 컬럼명 '오늘일자'를 '기준일자'로 변경하고, 컬럼 순서를 조정합니다.
new_column_order_graph_result = ['기준일자', '유형', '연관검색어', '검색일자', '검색량']

# 컬럼명 변경을 반영합니다. 실제 환경에서 실행할 때, graph_result가 정의되어 있어야 합니다.
graph_result.rename(columns={'오늘일자': '기준일자'}, inplace=True)

# 새로운 컬럼 순서로 데이터프레임을 재구성합니다.
graph_result = graph_result[new_column_order_graph_result]

# 결과 확인
graph_result.head(10)

,기준일자,유형,연관검색어,검색일자,검색량
0,2024-03-11,주별급상승,코인거래소,2020-03-10,2.42290
1,2024-03-11,주별급상승,코인거래소,2020-03-17,2.23935
2,2024-03-11,주별급상승,코인거래소,2020-03-24,2.73494
3,2024-03-11,주별급상승,코인거래소,2020-03-31,2.42290
4,2024-03-11,주별급상승,코인거래소,2020-04-07,2.40455
5,2024-03-11,주별급상승,코인거래소,2020-04-14,2.18428
6,2024-03-11,주별급상승,코인거래소,2020-04-21,2.36784
7,2024-03-11,주별급상승,코인거래소,2020-04-28,2.07415
8,2024-03-11,주별급상승,코인거래소,2020-05-05,1.54185
9,2024-03-11,주별급상승,코인거래소,2020-05-12,2.53303


# 구글/ 네이버 한꺼번에

### 오류가떠도 재시도를 하기위한 오류이기때문에 놀라지말기!

### 구글 검색어 

In [44]:
import asyncio
import time
from pytrends.request import TrendReq
from concurrent.futures import ThreadPoolExecutor

# Google Trends에 대한 비동기 조회를 위한 함수

# 비동기 작업을 위한 ThreadPoolExecutor 인스턴스 생성
executor = ThreadPoolExecutor(max_workers=5)

async def fetch_rising_queries(keyword: str, max_retries: int = 5) -> dict:
    pytrends = TrendReq(hl='ko-KR', tz=540, retries=2)
    kw_list = [keyword]

    for attempt in range(max_retries):
        try:
            loop = asyncio.get_running_loop()
            # Google Trends payload를 비동기적으로 구축
            await loop.run_in_executor(executor, lambda: pytrends.build_payload(kw_list, timeframe='today 1-m', geo='KR', gprop=''))
            # 관련 검색어 조회를 비동기적으로 실행
            result = await loop.run_in_executor(executor, pytrends.related_queries)
            rising_queries = result[keyword]['rising']
            if rising_queries is not None:
                return {keyword: list(rising_queries['query'])}
            return {keyword: []}
        except Exception as e:
            print(f"Error fetching data for {keyword}: {e}")
            # 지수 백오프를 사용한 재시도 대기
            await asyncio.sleep(2 ** attempt)
    return {keyword: []}  # 최대 재시도 후 빈 결과 반환

async def collect_rising_keywords(target_keywords: list) -> dict:
    # 주어진 키워드 리스트에 대해 비동기 조회 작업 생성
    tasks = [fetch_rising_queries(keyword) for keyword in target_keywords]
    # 모든 작업의 완료를 기다림
    results = await asyncio.gather(*tasks)
    aggregated_results = {}
    for result in results:
        aggregated_results.update(result)
    return aggregated_results

# 사용할 키워드 리스트 예시 (실제 사용시 info_result_final에서 추출)
target_keywords = list(info_result_final['연관키워드'])

# 비동기 함수 실행하여 결과 수집
rising_keywords_results = asyncio.run(collect_rising_keywords(target_keywords))
google_trend = rising_keywords_results

Error fetching data for 금가격: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22%5Cuae08%5Cuac00%5Cuaca9%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))
Error fetching data for 순금: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22%5Cuc21c%5Cuae08%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))
Error fetching data for 현재금값: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22

##### 뉴스링크

In [45]:
import asyncio
import aiohttp
from difflib import SequenceMatcher
import nest_asyncio

nest_asyncio.apply()

def clean_text(text):
    """
    HTML 태그 제거 및 특수 문자 처리 함수
    """
    text = text.replace('<b>', ' ').replace('</b>', ' ')
    text = text.replace('&quot;', '"').replace('&apos;', '\'')
    text = text.replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')
    return text

async def fetch_news(session, search_term, headers, params, attempts=10):
    """
    네이버 API를 통해 뉴스 데이터를 비동기적으로 가져오는 함수
    """
    backoff = 1
    for attempt in range(attempts):
        try:
            async with session.get("https://openapi.naver.com/v1/search/news.json", headers=headers, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['items']
                else:
                    raise Exception(f"Error fetching data for {search_term}: {response.status}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < attempts - 1:
                await asyncio.sleep(3 * backoff)  # 실패 시 지수적 백오프 적용 -> 오래걸리면 지수말고 고정/선형
                backoff *= 2
            else:
                raise

async def news_result_async(names):
    """
    검색어별 뉴스 제목, 본문, 링크 추출 및 필터링하는 비동기 함수
    """
    client_id = "ByXmMvAqMIxyVUY_h17L"  # 네이버 API 클라이언트 ID
    client_secret = "2x7yByvNSN"  # 네이버 API 클라이언트 Secret
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_news(session, name, headers, {"query": name, "display": 100, "sort": "date"}) for name in names]
        results = await asyncio.gather(*tasks)
        
        results_dict = {}
        for name, result in zip(names, results):
            titles_links = [(clean_text(item['title']), item['link']) for item in result[:10] if result]
            results_dict[name] = titles_links
            
    return results_dict

async def process_keywords_batch(batch):
    """
    키워드 배치 처리를 위한 함수
    """
    return await news_result_async(batch)

async def main(keywords):
    """
    비동기 실행을 위한 메인 함수, 키워드 리스트를 분할하여 처리
    """
    batch_size = 20 # 한 번에 처리할 키워드 수 # 20 : 160
    batches = [keywords[i:i + batch_size] for i in range(0, len(keywords), batch_size)]
    results = []

    for batch in batches:
        results.append(await process_keywords_batch(batch))
    
    # 결과 병합
    merged_results = {}
    for batch_result in results:
        merged_results.update(batch_result)

    return merged_results

# 실행
news_data = asyncio.run(main(target_keywords))

for keyword, data in news_data.items():
    print(f"Keyword: {keyword}")
    for title, link in data:
        print(f"Title: {title}, Link: {link}")


Attempt 1 failed: Error fetching data for AMD주가: 429
Attempt 1 failed: Error fetching data for 마이크로소프트주가: 429
Attempt 1 failed: Error fetching data for 애플주식: 429
Attempt 1 failed: Error fetching data for 도지코인시세: 429
Attempt 1 failed: Error fetching data for 해외주식: 429
Attempt 1 failed: Error fetching data for S&P500ETF: 429
Attempt 1 failed: Error fetching data for 샘플: 429
Attempt 1 failed: Error fetching data for 아이큐코인: 429
Attempt 1 failed: Error fetching data for 퇴직연금수령방법: 429
Attempt 1 failed: Error fetching data for 간병인보험: 429
Attempt 1 failed: Error fetching data for 삼성생명주가: 429
Attempt 1 failed: Error fetching data for 건강보험종류: 429
Attempt 1 failed: Error fetching data for 암보험비갱신형: 429
Attempt 1 failed: Error fetching data for 알테오젠주가: 429
Attempt 1 failed: Error fetching data for 보험상담: 429
Attempt 1 failed: Error fetching data for 암보험: 429
Attempt 1 failed: Error fetching data for 금값시세: 429
Attempt 1 failed: Error fetching data for 오늘금시세: 429
Attempt 1 failed: Error fetching data 

In [46]:

name_list = list(news_data.keys())  
# DataFrame 초기화
news_df = pd.DataFrame()

# 모든 키워드에 대해 처리
for keyword in name_list:
    # 뉴스 항목이 있는 경우 데이터 추가
    for news_item in news_data[keyword]:
        news_row = [keyword, news_item[0], news_item[1]]  # 연관키워드, 뉴스제목, 뉴스링크
        news_df = pd.concat([news_df, pd.DataFrame([news_row])], ignore_index=True)

    # 뉴스 항목 수가 10개에 미치지 못하면 나머지를 빈 행으로 채움
    for _ in range(10 - len(news_data[keyword])):
        empty_row = [keyword, None, None]  # 연관키워드, 빈 뉴스제목, 빈 뉴스링크
        news_df = pd.concat([news_df, pd.DataFrame([empty_row])], ignore_index=True)

# 칼럼 이름 설정
news_df.columns = ['연관검색어', '뉴스제목', '뉴스링크']
news_df.head()


,연관검색어,뉴스제목,뉴스링크
0,중고차어플,오퀴즈 'SK렌터카 다이렉트 중고차 신년맞이 최대할인' 7시 정답은?,http://www.joygm.com/news/articleView.html?idx...
1,중고차어플,"[스타트업 단신] 브리즘, 크림, 머니스테이션, 차봇 모빌리티, 구름, 프리윌린...",https://platum.kr/archives/221173
2,중고차어플,"[스타트업 단신] 오픈플랜, 나우로보틱스, 시소, 로지브라더스, 클로봇, 코코...",https://platum.kr/archives/219939
3,중고차어플,생애최초 청년창업 지원사업 참여 기업의 아이디어와 기술 선보이는 데모데이...,https://platum.kr/archives/219443
4,중고차어플,"카베이, 장기렌트 리스 전문 어플 차나와 출시",http://www.enetnews.co.kr/news/articleView.htm...


활동성 지표 삽입

In [47]:

with open('target_keywords.txt', 'w') as file:
    for keyword in target_keywords:
        file.write("%s\n" % keyword)

models/naver/blog.py 실행

In [48]:
%run models/naver/blog.py

Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A4%91%EA%B3%A0%EC%B0%A8%EC%96%B4%ED%94%8C&display=100&start=1&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A4%91%EA%B3%A0%EC%B0%A8%EC%96%B4%ED%94%8C&display=100&start=301&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A4%91%EA%B3%A0%EC%B0%A8%EC%96%B4%ED%94%8C&display=100&start=201&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A4%91%EA%B3%A0%EC%B0%A8%EC%96%B4%ED%94%8C&display=100&start=101&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A4%91%EA%B3%A0%EC%B0%A8%EC%96%B4%ED%94%8C&display=100&start=801&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A4%91%EA%B3%A0%EC%B0%A8%EC%96%B4%ED%94%8C&display=100&start=401&sort=date
Successfully fetched data for URL: https:/

In [104]:
keyword_activity_rates = pd.read_csv('keyword_activity_rates.csv')
keyword_activity_rates.columns = ['연관검색어', '활동성']

# '활동성' 열의 데이터를 백분율 형태의 문자열로 변환
keyword_activity_rates['활동성'] = keyword_activity_rates['활동성'].apply(lambda x: f"{x}%")
# news_df와 keyword_activity_rates를 '연관검색어' 열을 기준으로 병합
keyword_activity_rates = keyword_activity_rates.drop_duplicates(subset=['연관검색어'])
merged_keyword_activity_rates = pd.merge(news_df, keyword_activity_rates, on='연관검색어', how='left')

merged_keyword_activity_rates

,연관검색어,뉴스제목,뉴스링크,활동성
0,중고차어플,오퀴즈 'SK렌터카 다이렉트 중고차 신년맞이 최대할인' 7시 정답은?,http://www.joygm.com/news/articleView.html?idx...,0.0%
1,중고차어플,"[스타트업 단신] 브리즘, 크림, 머니스테이션, 차봇 모빌리티, 구름, 프리윌린...",https://platum.kr/archives/221173,0.0%
2,중고차어플,"[스타트업 단신] 오픈플랜, 나우로보틱스, 시소, 로지브라더스, 클로봇, 코코...",https://platum.kr/archives/219939,0.0%
3,중고차어플,생애최초 청년창업 지원사업 참여 기업의 아이디어와 기술 선보이는 데모데이...,https://platum.kr/archives/219443,0.0%
4,중고차어플,"카베이, 장기렌트 리스 전문 어플 차나와 출시",http://www.enetnews.co.kr/news/articleView.htm...,0.0%
...,...,...,...,...
3565,신용카드발급,"사회초년생 신용카드 , 소비성향에 맞춰 선택…이용한도 정해야 과다지출 방지",https://n.news.naver.com/mnews/article/662/000...,0.3%
3566,신용카드발급,해외소비 늘어 해외 전용 카드 '열풍'... 국내 혜자 카드 는 축소,http://www.g-enews.com/ko-kr/news/article/news...,0.3%
3567,신용카드발급,12일 신용 사면 앞두고 카드 사 문의 폭발…평균 39점 오른다,https://www.etoday.co.kr/news/view/2338589,0.3%
3568,신용카드발급,삼성 카드 ‘삼성식 상생금융’으로 지속가능 대책 마련 [상생하는 카드 사],http://www.fntimes.com/html/view.php?ud=202403...,0.3%


네이버 검색어 merge

In [105]:
collected_keywords_dat_copy.rename(columns={'연관키워드': '연관검색어'}, inplace=True)
# collected_keywords_dat_copy에서 '연관키워드'와 '검색어'를 기준으로 중복 제거
collected_keywords_dat_copy = collected_keywords_dat_copy.drop_duplicates(subset=['연관검색어'], keep='first')
# 이제 merged_keyword_activity_rates와 결합
final_merged_df = pd.merge(merged_keyword_activity_rates, collected_keywords_dat_copy[['연관검색어', '검색어']], on='연관검색어', how='left')



In [106]:
final_merged_df['구글검색어'] = None  # '구글검색어' 컬럼을 초기화합니다.

i = 0  # final_merged_df의 인덱스를 추적합니다.
for keyword, queries in news_data.items():
    # 각 키워드에 대한 상위 10개 쿼리를 가져옵니다. 쿼리가 10개 미만인 경우, 나머지는 None으로 채웁니다.
    filled_queries = queries[:10] + [None] * (10 - len(queries[:10]))
    
    # filled_queries의 각 쿼리에 대해 final_merged_df의 '구글검색어' 컬럼에 할당합니다.
    for query in filled_queries:
        if i < len(final_merged_df):  # i가 final_merged_df의 행 개수를 넘지 않도록 합니다.
            final_merged_df.at[i, '구글검색어'] = query
            i += 1
        else:
            break  # final_merged_df의 모든 행에 쿼리를 할당한 후에는 루프를 중단합니다.


# final_merged_df의 '검색어' 컬럼에서 각 10번째 검색어를 추출합니다.
keyword_list_per_10 = final_merged_df['검색어'].tolist()[::10]


 
# collected_keywords_dat_copy에서 각 검색어별 상위 10개 연관검색어를 가져옵니다.
# 여기서는 각 검색어별로 가장 높은 월간검색수를 가진 상위 10개를 선정합니다.
top_keywords_by_search = collected_keywords_dat_copy.groupby('검색어').apply(
    lambda x: x.nlargest(10, '월간검색수_합계')
).reset_index(drop=True)



# 새로운 DataFrame을 초기화합니다. 이 DataFrame에는 각 검색어별 상위 10개 연관검색어가 포함됩니다.
new_rows_for_final_df = []



# final_merged_df에서 추출한 검색어 리스트를 순회하며, 각 검색어별 상위 10개 연관검색어를 new_rows_for_final_df에 추가합니다.
for keyword in keyword_list_per_10:
    top_queries_for_keyword = top_keywords_by_search[top_keywords_by_search['검색어'] == keyword].head(10)
    for _, row in top_queries_for_keyword.iterrows():
        new_rows_for_final_df.append(row['연관검색어'])



# new_rows_for_final_df의 길이를 확인하고 final_merged_df의 '네이버검색어' 컬럼에 값을 할당합니다.
# 주의: new_rows_for_final_df의 길이가 final_merged_df의 행 수와 동일해야 합니다.
# 만약 길이가 다르다면, 길이가 맞도록 조정이 필요합니다.
if len(new_rows_for_final_df) == len(final_merged_df):
    final_merged_df['네이버검색어'] = new_rows_for_final_df
else:
    print("경고: '네이버검색어' 데이터의 길이가 final_merged_df와 다릅니다. 데이터 확인이 필요합니다.")

# 최종 DataFrame 확인
final_merged_df

C:\Users\chohy\AppData\Local\Temp\ipykernel_12148\3122009593.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_keywords_by_search = collected_keywords_dat_copy.groupby('검색어').apply(


,연관검색어,뉴스제목,뉴스링크,활동성,검색어,구글검색어,네이버검색어
0,중고차어플,오퀴즈 'SK렌터카 다이렉트 중고차 신년맞이 최대할인' 7시 정답은?,http://www.joygm.com/news/articleView.html?idx...,0.0%,리워드,"(오퀴즈 'SK렌터카 다이렉트 중고차 신년맞이 최대할인' 7시 정답은?, htt...",산티아고순례길
1,중고차어플,"[스타트업 단신] 브리즘, 크림, 머니스테이션, 차봇 모빌리티, 구름, 프리윌린...",https://platum.kr/archives/221173,0.0%,리워드,"([스타트업 단신] 브리즘, 크림, 머니스테이션, 차봇 모빌리티, 구름, 프리윌린....",단기알바
2,중고차어플,"[스타트업 단신] 오픈플랜, 나우로보틱스, 시소, 로지브라더스, 클로봇, 코코...",https://platum.kr/archives/219939,0.0%,리워드,"([스타트업 단신] 오픈플랜, 나우로보틱스, 시소, 로지브라더스, 클로봇, 코코.....",아르바이트
3,중고차어플,생애최초 청년창업 지원사업 참여 기업의 아이디어와 기술 선보이는 데모데이...,https://platum.kr/archives/219443,0.0%,리워드,"(생애최초 청년창업 지원사업 참여 기업의 아이디어와 기술 선보이는 데모데이..., ...",데이터라벨링
4,중고차어플,"카베이, 장기렌트 리스 전문 어플 차나와 출시",http://www.enetnews.co.kr/news/articleView.htm...,0.0%,리워드,"(카베이, 장기렌트 리스 전문 어플 차나와 출시, http://www.enetn...",알디콤
...,...,...,...,...,...,...,...
3565,신용카드발급,"사회초년생 신용카드 , 소비성향에 맞춰 선택…이용한도 정해야 과다지출 방지",https://n.news.naver.com/mnews/article/662/000...,0.3%,캐시백,"(사회초년생 신용카드 , 소비성향에 맞춰 선택…이용한도 정해야 과다지출 방지, h...",신용카드발급
3566,신용카드발급,해외소비 늘어 해외 전용 카드 '열풍'... 국내 혜자 카드 는 축소,http://www.g-enews.com/ko-kr/news/article/news...,0.3%,캐시백,"(해외소비 늘어 해외 전용 카드 '열풍'... 국내 혜자 카드 는 축소, htt...",신세계백화점카드
3567,신용카드발급,12일 신용 사면 앞두고 카드 사 문의 폭발…평균 39점 오른다,https://www.etoday.co.kr/news/view/2338589,0.3%,캐시백,"(12일 신용 사면 앞두고 카드 사 문의 폭발…평균 39점 오른다, https:...",카드이벤트
3568,신용카드발급,삼성 카드 ‘삼성식 상생금융’으로 지속가능 대책 마련 [상생하는 카드 사],http://www.fntimes.com/html/view.php?ud=202403...,0.3%,캐시백,"(삼성 카드 ‘삼성식 상생금융’으로 지속가능 대책 마련 [상생하는 카드 사], ...",카드사


# info_result_out

In [107]:
from datetime import datetime
info_result_final_unique = info_result_final.drop_duplicates(subset=['연관키워드'])

repeated_rows = []

# info_result_final_unique의 각 행에 대해 10번 반복하여 repeated_rows에 추가합니다.
for _, row in info_result_final_unique.iterrows():
    for _ in range(10):
        repeated_rows.append(row.values.tolist())

# 새로운 DataFrame 생성
info_result_final_repeated = pd.DataFrame(repeated_rows, columns=info_result_final_unique.columns)
# 필요한 컬럼만 info_result_final_repeated에서 추출합니다.
columns_to_add = ['월간검색수_합계', '중복검색어', '유형', '지표', '상승월']

# final_merged_df에 컬럼을 추가합니다. 이 때, info_result_final_repeated의 각 컬럼을 직접 할당합니다.
for col in columns_to_add:
    final_merged_df[col] = info_result_final_repeated[col].values


# 오늘 날짜 설정
today_date = datetime.now().strftime("%Y-%m-%d")

# '기준일자' 컬럼을 가장 앞에 추가
final_merged_df.insert(0, '기준일자', today_date)

# 컬럼 순서를 조정하고, '검색어' 컬럼을 제거합니다.
new_column_order = ['기준일자', '유형', '연관검색어', '중복검색어', '월간검색수_합계', '지표', '뉴스제목', '뉴스링크', '활동성', '구글검색어', '네이버검색어', '상승월']

# 컬럼명 변경: '중복검색어' -> '검색키워드', '월간검색수_합계' -> '검색량'
info_result_af = final_merged_df[new_column_order]
info_result_af.rename(columns={'중복검색어': '검색키워드', '월간검색수_합계': '검색량'}, inplace=True)
info_result_af['상승월'] = None

# rising_month_list의 각 항목에 대해 반복 처리
for month_info in rising_month_list:
    months, keyword = month_info  # month_info는 각각의 월 목록과 키워드를 포함합니다.
    keyword_rows = info_result_af[info_result_af['연관검색어'] == keyword]  # 해당 키워드에 대한 행만 선택합니다.
    
    if not keyword_rows.empty:

        for i, month in enumerate(months):
            if i < len(keyword_rows):
                info_result_af.loc[keyword_rows.index[i], '상승월'] = month
            else:
                break  # 월의 개수보다 더 많은 행에 대해서는 처리를 중단합니다.


C:\Users\chohy\AppData\Local\Temp\ipykernel_12148\993304406.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_result_af.rename(columns={'중복검색어': '검색키워드', '월간검색수_합계': '검색량'}, inplace=True)
C:\Users\chohy\AppData\Local\Temp\ipykernel_12148\993304406.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_result_af['상승월'] = None


In [108]:
# graph_result 데이터프레임의 컬럼명 '오늘일자'를 '기준일자'로 변경하고, 컬럼 순서를 조정합니다.
new_column_order_graph_result = ['기준일자', '유형', '연관검색어', '검색일자', '검색량']

# 컬럼명 변경을 반영합니다. 실제 환경에서 실행할 때, graph_result가 정의되어 있어야 합니다.
graph_result.rename(columns={'오늘일자': '기준일자'}, inplace=True)

# 새로운 컬럼 순서로 데이터프레임을 재구성합니다.
graph_result = graph_result[new_column_order_graph_result]

# 결과 확인
(graph_result.head())

,기준일자,유형,연관검색어,검색일자,검색량
0,2024-03-11,주별급상승,코인거래소,2020-03-10,2.42290
1,2024-03-11,주별급상승,코인거래소,2020-03-17,2.23935
2,2024-03-11,주별급상승,코인거래소,2020-03-24,2.73494
3,2024-03-11,주별급상승,코인거래소,2020-03-31,2.42290
4,2024-03-11,주별급상승,코인거래소,2020-04-07,2.40455


In [123]:
column_names = '|||'.join(info_result_af.columns)

# 각 행을 '|||'로 구분된 문자열로 변환하고 새로운 DataFrame의 한 컬럼으로 저장
df_string = info_result_af.apply(lambda x: '|||'.join(x.astype(str)), axis=1)
new_df = pd.DataFrame(df_string, columns=[column_names])

today_date = datetime.now().strftime("%y%m%d")

base_save_path = "./data/result_out"

# 오늘 날짜를 포함한 최종 저장 경로 생성
final_save_path = os.path.join(base_save_path, today_date)

# 최종 저장 경로가 없는 경우 생성
if not os.path.exists(final_save_path):
    os.makedirs(final_save_path)

# 파일 저장 경로 설정
file_save_path = os.path.join(final_save_path, f"info_result_out_{today_date}.csv")

# DataFrame을 CSV 파일로 저장
new_df.to_csv(file_save_path, index=False, encoding='utf-8-sig')
print(f"File saved at: {file_save_path}")

File saved at: ./data/result_out\240311\info_result_out_240311.csv


In [124]:
column_names = '|||'.join(graph_result.columns[:-1]) + '|||'


# 각 행을 '|||'로 구분된 문자열로 변환하고 새로운 DataFrame의 한 컬럼으로 저장
df_string = graph_result.apply(lambda x: '|||'.join(x.astype(str)), axis=1)
new_df = pd.DataFrame(df_string, columns=[column_names])

today_date = datetime.now().strftime("%y%m%d")

base_save_path = "./data/result_out"

# 오늘 날짜를 포함한 최종 저장 경로 생성
final_save_path = os.path.join(base_save_path, today_date)

# 최종 저장 경로가 없는 경우 생성
if not os.path.exists(final_save_path):
    os.makedirs(final_save_path)

# 파일 저장 경로 설정
file_save_path = os.path.join(final_save_path, f"graph_result_out_{today_date}.csv")

# DataFrame을 CSV 파일로 저장
new_df.to_csv(file_save_path, index=False, encoding='utf-8-sig')
print(f"File saved at: {file_save_path}")

File saved at: ./data/result_out\240311\graph_result_out_240311.csv
